# Задание 1

Есть таблица с действиями юзеров A/B-теста вида:
- 𝑒𝑣𝑒𝑛𝑡𝑠  - установки приложения
- 𝑢𝑠𝑒𝑟_𝑖𝑑  - id юзера,
- 𝑎𝑏_𝑔𝑟𝑜𝑢𝑝  - группа A/B-теста,
- 𝑡𝑠  - время совершения действия,
- 𝑝𝑑𝑎𝑡𝑒  - дата совершения действия.

Пользовательская сессия определяется по следующим правилам:

- Новая сессия начинается после 30 минут бездействия.
- Сессия прерывается при переходе между двумя датами.

Необходимо построить таблицу с сессиями юзеров в формате:

- 𝑢𝑠𝑒𝑟_𝑖𝑑  - id юзера
- 𝑎𝑏_𝑔𝑟𝑜𝑢𝑝  - группа A/B-теста,
- 𝑠𝑡𝑎𝑟𝑡_𝑡𝑠  - время старта сессии,
- 𝑒𝑛𝑑_𝑡𝑠  - время окончания сессии,
- 𝑝𝑑𝑎𝑡𝑒  - дата сессии.


In [13]:
# Для начала импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, norm

In [2]:
# Считаем файл
df = pd.read_csv('task_1_events.csv', sep=",")
df.head(3)

,user_id,ab_group,ts,pdate
0,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:24:33.553327,2023-05-02
1,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:32:23.215284,2023-05-02
2,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:33:01.050395,2023-05-02


In [3]:
df.dtypes

user_id     object
ab_group    object
ts          object
pdate       object
dtype: object

Как мы видим, все данные в формате object, для дальнейшей работы стоит перевести данные с датой в формат datetime

In [4]:
# Преобразуем столбцы 'ts' и 'pdate' в тип данных datetime
df['ts'] = pd.to_datetime(df['ts'])
df['pdate'] = pd.to_datetime(df['pdate'])

In [5]:
df.dtypes

user_id             object
ab_group            object
ts          datetime64[ns]
pdate       datetime64[ns]
dtype: object

In [6]:
# Отсортируем данные по 'user_id' и 'ts'
df.sort_values(['user_id', 'ts'], inplace=True)

# Создадим столбец 'время_бездействия', который показывает разницу в минутах между текущим и предыдущим временем 'тс' для каждого пользователя
df['no_action_time'] = df.groupby('user_id')['ts'].diff().dt.total_seconds() / 60

# Создадим столбец 'новая_сессия', который указывает, является ли текущее действие началом новой сессии (30 минут бездействия и прерывание при смене дат)
df['new_session'] = (df['no_action_time'].fillna(0) > 30) | (df['pdate'].diff().dt.days > 0)

# Создадим столбец 'сессия', который будет увеличиваться на 1 с каждой новой сессией для каждого пользователя
df['session'] = df.groupby('user_id')['new_session'].cumsum()

# Группируем данные по 'user_id', 'session' и 'pdate', и агрегируйте значения минимального и максимального времени 'ts'
sessions_df = df.groupby(['user_id', 'session', 'pdate', 'ab_group']).agg({'ts': ['min', 'max']})

# Переименуем столбцы
sessions_df.columns = ['start_ts', 'end_ts']

# Сбросим индексы, чтобы было красиво
sessions_df.reset_index(inplace=True)

# Удалим столбцы, которые больше не нужны
sessions_df.drop(['session'], axis=1, inplace=True)

# Перенесем столбец 'pdate' в конец таблицы,  чтобы соответствовало тз
sessions_df = sessions_df[['user_id', 'ab_group', 'start_ts', 'end_ts', 'pdate']]

# Отсортируем по user_id, чтобы увидеть все сессии у пользователя
sessions_df.sort_values('user_id', inplace=True)

# Выведем таблицу сессий пользователей в соответствии с тз
sessions_df.head(5)


,user_id,ab_group,start_ts,end_ts,pdate
0,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:45.766570,2023-05-01 08:57:37.363424,2023-05-01
1,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:24:33.553327,2023-05-02 12:37:57.761142,2023-05-02
2,3E69ZO0RJrEWACi28XaTPg==,A,2023-05-01 18:58:45.338597,2023-05-01 19:04:19.203838,2023-05-01
3,3E69ZO0RJrEWACi28XaTPg==,A,2023-05-04 10:01:39.700496,2023-05-04 10:07:39.858666,2023-05-04
4,3GsYmewRRswWABqG8XaTPg==,B,2023-05-03 05:00:41.797222,2023-05-03 05:18:12.137282,2023-05-03


# Задание 2

### Текст задания

Был проведен A/B-тест.
В качестве данных используйте таблицу, построенную в предыдущем задании. Первая сессия юзера считается моментом попадания в A/B-тест.

Ключевая метрика эксперимента - конверсия во вторую сессию.
Сделайте вывод о том, какая группа выиграла в A/B-тесте. Ответ обоснуйте.

### Решение

Проверим данные

In [7]:
#Проверим, есть ли пользователи, которые попали в обе тестовых группы
double_variant_count = sessions_df.groupby('user_id')['ab_group'].nunique().value_counts()
double_variant_count

1    5000
Name: ab_group, dtype: int64

Пользователей, которые бы попали в обе группы - нет, продолжаем без изменений

Для начала посчитаем конверсию во вторую сессию для каждой из групп теста

In [8]:
# Определим первую и вторую сессии каждого пользователя в A/B-тесте
first_sessions = sessions_df.groupby(['user_id', 'ab_group'])['start_ts'].min()
second_sessions = sessions_df.groupby(['user_id', 'ab_group'])['start_ts'].nth(1)

# Подсчитаем количество пользователей для каждой группы A/B-теста
users_count = first_sessions.groupby('ab_group').count()

# Подсчитаем количество пользователей с второй сессией для каждой группы A/B-теста
users_with_second_session = second_sessions.groupby('ab_group').count()

# Рассчитаем конверсию во вторую сессию для каждой группы A/B-теста
conversion = users_with_second_session / users_count

# Выведем результаты
print(conversion)

ab_group
A    0.1272
B    0.1412
Name: start_ts, dtype: float64


Мы видим, что вариант B лидирует, но этого недостаточно, чтобы сделать выводы. Посчитаем с помощью статистичесиких методов статистическую значимость:

Используем хи-квадрат

Тест хи-квадрат подходит для анализа категориальных данных и позволяет определить, есть ли статистически значимая связь между двумя или более переменными. В нашем случае мы используем его для сравнения пропорций (конверсий) в двух группах и определения, есть ли статистически значимая разница между ними.

Выбор теста зависит от типа данных и цели исследования. В данном случае, когда мы имеем дело с категориальными данными и интересует разница между пропорциями, тест хи-квадрат является одним из подходящих вариантов для статистического анализа.

In [9]:
# Загрузим полученные данные в переменные
observed = np.array([[users_with_second_session['A'], users_count['A']],
                     [users_with_second_session['B'], users_count['B']]])

# Проведем тест по хи-квадрат
chi2, p, d, ex = chi2_contingency((observed), correction=False)

# Рассчитаем доли успеха (конверсии) в каждом варианте
conversion_rate_A = users_with_second_session['A'] / users_count['A']
conversion_rate_B = users_with_second_session['B'] / users_count['B']

# Рассчитаем доверительные интервалы для каждого варианта
se_A = np.sqrt(conversion_rate_A * (1 - conversion_rate_A) / users_count['A'])
se_B = np.sqrt(conversion_rate_B * (1 - conversion_rate_B) / users_count['B'])
ci_low_A = conversion_rate_A - 1.96 * se_A
ci_high_A = conversion_rate_A + 1.96 * se_A
ci_low_B = conversion_rate_B - 1.96 * se_B
ci_high_B = conversion_rate_B + 1.96 * se_B

# Выведем результаты
print("p-value:", p)
print("Доверительный интервал для варианта A:", np.round(ci_low_A, 3), np.round(ci_high_A, 3))
print("Доверительный интервал для варианта B:", np.round(ci_low_B, 3), np.round(ci_high_B, 3))

# Проверим, есть ли изменения
if p < 0.05 and ci_low_B > ci_high_A:
    print("Вариант B лучше.")
else:
    print("Разница между вариантами не является статистически значимой.")


p-value: 0.20453631043808843
Доверительный интервал для варианта A: 0.114 0.14
Доверительный интервал для варианта B: 0.128 0.155
Разница между вариантами не является статистически значимой.


На основании результатов, которые мы получили, можно сделать следующие выводы:

- Значение p-value равно 0.2045, что выше уровня значимости 0.05. Это означает, что нет статистически значимой разницы в конверсии между вариантами A и B.

- Доверительные интервалы для обоих вариантов пересекаются. Это также указывает на отсутствие статистически значимой разницы в конверсии между вариантами.

Проверим верность решения на другом статистическом методе. Посмотрим, является ли различие статзначимым при расчете z-test

In [14]:
# Загрузим полученные данные в переменные
conversion_A = conversion['A']
conversion_B = conversion['B']
users_count_A = users_count['A']
users_count_B = users_count['B']

# Рассчитаем стандартное отклонение для каждого варианта
std_A = np.sqrt(conversion_A * (1 - conversion_A) / users_count_A)
std_B = np.sqrt(conversion_B * (1 - conversion_B) / users_count_B)

# Рассчитаем Z-статистику
z_score = (conversion_B - conversion_A) / np.sqrt(std_A**2 + std_B**2)

# Рассчитаем p-value
p_value = 1 - stats.norm.cdf(z_score)

# Определим доверительный уровень значимости
alpha = 0.05

# Проверим статистическую значимость
if p_value < alpha:
    print("Разница между вариантами статистически значима.")
else:
    print("Разница между вариантами не является статистически значимой.")

# Выведем значения Z-статистики и p-value
print("Z-статистика:", z_score)
print("p-value:", p_value)

Разница между вариантами не является статистически значимой.
Z-статистика: 1.4524111977190173
p-value: 0.07319365229452202


На основании результатов теста Z-теста, мы получаем следующие результаты:

- Разница между вариантами A и B не является статистически значимой, так как p-value (0.073) превышает выбранный уровень значимости alpha (в нашем случае = 0.05).

- Значение Z-статистики (1.452) показывает направление разницы между вариантами A и B, где положительное значение указывает на то, что конверсия во вторую сессию варианта B может быть выше, но это значение не достаточно сильное, чтобы считать разницу статистически значимой.

Таким образом, на основе проведенного Z-теста мы также не можем сделать вывод о статистически значимой разнице между конверсией во вторую сессию вариантов A и B

Итоговые выводы и рекомендации:

- Основываясь на имеющихся данных, нет оснований для того, чтобы считать, что вариант A или вариант B лучше с точки зрения конверсии во вторую сессию.

- Рекомендуется провести дальнейшие исследования или собрать больше данных, чтобы получить более точные и надежные результаты. Возможно, изменение факторов или продолжение тестирования может привести к обнаружению значимых различий.

- Важно также учитывать другие факторы, такие как пользовательский опыт, пользовательские отзывы или бизнес-цели при принятии решения о выборе варианта A или B.

- Не до конца понятно условие "сессия прерывается при переходе между двумя датами". Если сессия принудительно прерывается у всех пользователей в эту секунду, и появляется экран "сессия прервана", то возможно стоит провести новый расчет, и исключить всех пользователей, которые попали на этот перелом дат, потому что такая вторая сессия не в полной мере является полноцецнной и может смазать результаты эксперимента.